# Models

In [ ]:
import numpy as np
import sympy as sp

# parameters
class PAR:
    l1 = 1.0 # length of pendulum 1
    l2 = 1.0 # length of pendulum 2
    g = 9.81 # gravity
    μ1 = 0.6 # damping coefficient
    μ2 = 0.6 # damping coefficient
    m1 = .1 # mass of pendulum 1
    m2 = .1 # mass of pendulum 2
    mc = .1 # mass of cart

In [ ]:
SINGLE_PENDULUM_EQ = 'models/single_pendulum_eq.py'

class Model:
    def __init__(self, dt, x0=None):
        self.ss = [] if x0 is None else [x0] # state history
        self.us = [] # control history
        self.dt = dt # time step (continous time)
        
    def step(self, u): 
        '''
        Step the model forward in time
        u: control input
        '''
        raise NotImplementedError

def sp2np(f:sp.Function):
    '''sketchy way to convert sympy expression to numpy'''
    f = f.simplify()
    f = str(f)
    subs = [('u(t)', 'u')]
    for i in range(1, 7):
        subs.append((f'Derivative(s{i}(t), (t, 3))', f'ddds{i}'))
        subs.append((f'Derivative(s{i}(t), (t, 2))', f'dds{i}'))
        subs.append((f'Derivative(s{i}(t), (t, 1))', f'ds{i}'))
        subs.append((f'Derivative(s{i}(t), t)', f'ds{i}')) 
        subs.append((f's{i}(t)', f's{i}'))
    for s in subs: f = f.replace(*s)
    return f

def save_model(model, filename):
    '''save model to file'''
    pass

# Taylor approximation at s0 of the function 'f'
def taylor(f:sp.Function,s:sp.Symbol,s0:float,n:int):
    def factorial(n):
        if n <= 0: return 1
        else: return n*factorial(n-1)
    i, p = 0, 0
    while i <= n:
        p = p + (f.diff(s,i).subs(s,s0))/(factorial(i))*(s-s0)**i
        i += 1
    return p

def linear_system(f:sp.Function, states:list, inputs:list, s_eq:list):
    raise NotImplementedError
    # linearize the system around the equilibrium point s0
    # NOTE: states and their derivatives must be in the order with derivatives first
    n, m = len(states), len(inputs)
    A = sp.zeros(n,n)
    B = sp.zeros(n,m)
    for i in range(n):
        for j in range(n):
            aij = taylor(f, states[i], s_eq[i], 1)
            for k in range(n):
                if k == i: aij = aij.subs(states[k], 1)
                else: aij = aij.subs(states[k], 0) 
        for j in range(m):
            B[i,j] = s_eq[i].diff(inputs[j])
    


In [ ]:
# single pendulum
t = sp.symbols('t', real=True)
u = sp.symbols('u', cls=sp.Function, real=True)(t)

# single pendulum
l1, m1, g, μ1 = sp.symbols('l1 m1 g μ1', positive=True, real=True) 
s1 = sp.symbols('s1', cls=sp.Function, real=True)(t) 
ds1 = sp.diff(s1, t)
dds1 = sp.diff(ds1, t)

V = m1 * g * l1 * sp.cos(s1) # potential energy
T = 0.5 * m1 * (l1 * ds1)**2 # kinetic energy
L = T - V # lagrangian

print('\nLagrangian:')
print(sp2np(L))

L1 = L.diff(s1) - (L.diff(ds1)).diff(t) - μ1 * ds1 + u

print('\nEuler-Lagrange equation:')
print(sp2np(L1))

# solve
sol = sp.solve(L1, dds1, simplify=False)
dds1 = sol[0]

print('\nSolved:')
print(f'dds1 == {sp2np(dds1)}')

# linearize around s1=0
ldds1_s1 = taylor(dds1, s1, 0, 1).subs(u, 0).subs(ds1, 0).subs(s1, 1)
ldds1_ds1 = taylor(dds1, ds1, 0, 1).subs(u, 0).subs(ds1, 1).subs(s1, 0)
# ldds1_ds1 = taylor(dds1, ds1, 0, 1).subs(u, 0).subs(s1, 0).subs(ds1, 1) # wrong
print('\nLinearized around s1=0:')
print(f'A11 == {sp2np(ldds1_s1)}')
print(f'A12 == {sp2np(ldds1_ds1)}')

# sobstitute parameters
dds1 = dds1.subs([(l1, PAR.l1), (m1, PAR.m1), (g, PAR.g), (μ1, PAR.μ1)])
print('\nSubstituted parameters:')
print(sp2np(dds1))

In [ ]:
# test the integration
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def f(t, x, u):
    return x[1], eval(dds1)



In [ ]:
# double pendulum
l1, l2, m1, m2, g, μ1, μ2 = sp.symbols('l1 l2 m1 m2 g μ1 μ2', positive=True, real=True)
s1 = sp.symbols('s1', cls=sp.Function, real=True)(t)
s2 = sp.symbols('s2', cls=sp.Function, real=True)(t)
ds1, ds2 = sp.diff(s1, t), sp.diff(s2, t)
dds1, dds2 = sp.diff(ds1, t), sp.diff(ds2, t)
x1, y1 = l1 * sp.sin(s1), l1 * sp.cos(s1)
x2, y2 = x1 + l2 * sp.sin(s2), y1 + l2 * sp.cos(s2)
dx1, dy1 = sp.diff(x1, t), sp.diff(y1, t)
dx2, dy2 = sp.diff(x2, t), sp.diff(y2, t)

V = m1 * g * y1 + m2 * g * y2 # potential energy
T = 0.5 * m1 * (dx1**2 + dy1**2) + 0.5 * m2 * (dx2**2 + dy2**2) # kinetic energy
L = T - V # lagrangian

print('\nLagrangian:')
print(sp2np(L))

L1 = L.diff(s1) - (L.diff(ds1)).diff(t) - μ1 * ds1 + u
L2 = L.diff(s2) - (L.diff(ds2)).diff(t) - μ2 * ds2

print('\nEuler-Lagrange equation:')
print(sp2np(L1))
print(sp2np(L2))

# solve
sol = sp.solve([L1, L2], [dds1, dds2], simplify=False)
dds1, dds2 = sol[dds1], sol[dds2]

print('\nSolved:')
print(f'dds1 == {dds1}')
print(f'dds2 == {dds2}')
print(f'dds1 == {sp2np(dds1)}')
print(f'dds2 == {sp2np(dds2)}')

# linearize around s1=s2=0
ldds1_s1 = taylor(dds1, s1, 0, 1).subs(u, 0).subs(ds1, 0).subs(s1, 1).subs(ds2, 0).subs(s2, 0)
ldds1_ds1 = taylor(dds1, ds1, 0, 1).subs(u, 0).subs(ds1, 1).subs(s1, 0).subs(ds2, 0).subs(s2, 0)

ldds2_s2 = taylor(dds2, s2, 0, 1).subs(u, 0).subs(ds2, 0).subs(s2, 1).subs(ds1, 0).subs(s1, 0)
ldds2_ds2 = taylor(dds2, ds2, 0, 1).subs(u, 0).subs(ds2, 1).subs(s2, 0).subs(ds1, 0).subs(s1, 0)

print('\nLinearized around s1=s2=0:')
print(f'A11 == {sp2np(ldds1_s1)}')
print(f'A12 == {sp2np(ldds1_ds1)}')
print(f'A13 == {sp2np(ldds2_s2)}')
print(f'A14 == {sp2np(ldds2_ds2)}')

# sobstitute parameters
dds1 = dds1.subs([(l1, PAR.l1), (l2, PAR.l2), (m1, PAR.m1), (m2, PAR.m2), (g, PAR.g), (μ1, PAR.μ1), (μ2, PAR.μ2)])
dds2 = dds2.subs([(l1, PAR.l1), (l2, PAR.l2), (m1, PAR.m1), (m2, PAR.m2), (g, PAR.g), (μ1, PAR.μ1), (μ2, PAR.μ2)])
print('\nSubstituted parameters:')
print(sp2np(dds1))
print(sp2np(dds2))

In [ ]:
# cart single pendulum
l1, m1, m2, g, μ1 = sp.symbols('l1 m1 m2 g μ1', positive=True, real=True) 
s1 = sp.symbols('s1', cls=sp.Function, real=True)(t) # angle
s2 = sp.symbols('s2', cls=sp.Function, real=True)(t) # position
ds1, ds2 = sp.diff(s1, t), sp.diff(s2, t)
dds1, dds2 = sp.diff(ds1, t), sp.diff(ds2, t)
x1, y1 = l1 * sp.sin(s1) + s2, l1 * sp.cos(s1)
dx1, dy1 = sp.diff(x1, t), sp.diff(y1, t)

V = m1 * g * y1 # potential energy
T = 0.5 * m1 * (dx1**2 + dy1**2) + 0.5 * m2 * ds2**2 # kinetic energy
L = T - V # lagrangian

print('\nLagrangian:')
print(sp2np(L))

L1 = L.diff(s1) - (L.diff(ds1)).diff(t) - μ1 * ds1 
L2 = L.diff(s2) - (L.diff(ds2)).diff(t) + u/m2

print('\nEuler-Lagrange equation:')
print(sp2np(L1))
print(sp2np(L2))

# solve
sol = sp.solve([L1, L2], [dds1, dds2], simplify=False)
dds1, dds2 = sol[dds1], sol[dds2]

print('\nSolved:')
print(f'dds1 == {dds1}')
print(f'dds2 == {dds2}')
print(f'dds1 == {sp2np(dds1)}')
print(f'dds2 == {sp2np(dds2)}')

# linearize around s1=0
ldds1_s1 = taylor(dds1, s1, 0, 1).subs(u, 0).subs(ds1, 0).subs(s1, 1).subs(ds2, 0).subs(s2, 0)
ldds1_ds1 = taylor(dds1, ds1, 0, 1).subs(u, 0).subs(ds1, 1).subs(s1, 0).subs(ds2, 0).subs(s2, 0)
print('\nLinearized around s1=0:')
print(f'A11 == {sp2np(ldds1_s1)}')
print(f'A12 == {sp2np(ldds1_ds1)}')

# substitute the actual parameters
dds1 = dds1.subs({l1:PAR.l1, m1:PAR.m1, g:PAR.g, μ1:PAR.μ1})
dds2 = dds2.subs({l1:PAR.l1, m1:PAR.m1, g:PAR.g, μ1:PAR.μ1, m2:PAR.m2})

print('\nSubstituted:')
print(f'dds1 == {sp2np(dds1)}') 
print(f'dds2 == {sp2np(dds2)}')